In [ ]:
# Mount Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd /content/drive/My Drive/Github/Machine Learning Course Implementations

/content/drive/My Drive/Github/Machine Learning Course Implementations


In [ ]:
ls

client_test.csv                         Submission17.csv  Submission2.csv
client_train.csv                        Submission18.csv  Submission3.csv
colabgitWorkflow.ipynb                  Submission19.csv  Submission4.csv
invoice_test.csv                        Submission20.csv  Submission5.csv
invoice_train.csv                       Submission21.csv  Submission6.csv
logs.log                                Submission22.csv  Submission7.csv
Project1:FraudDetectionChallenge.ipynb  Submission23.csv  Submission8.csv
SampleSubmission.csv                    Submission24.csv  Submission9.csv
Submission16.csv                        Submission25.csv  Submission.csv


In [ ]:
#import needed libraries 

import numpy as np
import pandas as pd
import datetime
import gc
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import mean_squared_error
import warnings
warnings.filterwarnings('ignore')
np.random.seed(4590)
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier,ExtraTreeClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

from sklearn.linear_model import LinearRegression, Lasso, Ridge, LassoCV, RidgeCV, MultiTaskLassoCV, LassoLarsCV
from sklearn import datasets
from sklearn import preprocessing
from sklearn.model_selection import train_test_split, KFold, cross_val_score, GridSearchCV, LeaveOneOut
from sklearn import tree
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, plot_confusion_matrix
from sklearn.preprocessing import PolynomialFeatures

from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.feature_selection import RFE

# Nouvelle section

In [ ]:
train_client=pd.read_csv('./client_train.csv')
test_client=pd.read_csv('./client_test.csv')
train_invoice=pd.read_csv('./invoice_train.csv')
test_invoice=pd.read_csv('./invoice_test.csv')
sub=pd.read_csv('./SampleSubmission.csv')

In [ ]:
train_client.head()

,disrict,client_id,client_catg,region,creation_date,target
0,60,train_Client_0,11,101,31/12/1994,0.0
1,69,train_Client_1,11,107,29/05/2002,0.0
2,62,train_Client_10,11,301,13/03/1986,0.0
3,69,train_Client_100,11,105,11/07/1996,0.0
4,62,train_Client_1000,11,303,14/10/2014,0.0


In [ ]:
train_invoice.head(10)

,client_id,invoice_date,tarif_type,counter_number,counter_statue,counter_code,reading_remarque,counter_coefficient,consommation_level_1,consommation_level_2,consommation_level_3,consommation_level_4,old_index,new_index,months_number,counter_type
0,train_Client_0,2014-03-24,11,1335667,0,203,8,1,82,0,0,0,14302,14384,4,ELEC
1,train_Client_0,2013-03-29,11,1335667,0,203,6,1,1200,184,0,0,12294,13678,4,ELEC
2,train_Client_0,2015-03-23,11,1335667,0,203,8,1,123,0,0,0,14624,14747,4,ELEC
3,train_Client_0,2015-07-13,11,1335667,0,207,8,1,102,0,0,0,14747,14849,4,ELEC
4,train_Client_0,2016-11-17,11,1335667,0,207,9,1,572,0,0,0,15066,15638,12,ELEC
5,train_Client_0,2017-07-17,11,1335667,0,207,9,1,314,0,0,0,15638,15952,8,ELEC
6,train_Client_0,2018-12-07,11,1335667,0,207,9,1,541,0,0,0,15952,16493,12,ELEC
7,train_Client_0,2019-03-19,11,1335667,0,207,9,1,585,0,0,0,16493,17078,8,ELEC
8,train_Client_0,2011-07-22,11,1335667,0,203,9,1,1200,186,0,0,7770,9156,4,ELEC
9,train_Client_0,2011-11-22,11,1335667,0,203,6,1,1082,0,0,0,9156,10238,4,ELEC


In [ ]:
d={"ELEC":0,"GAZ":1}
train_invoice['counter_type']=train_invoice['counter_type'].map(d)

In [ ]:
train_client['client_catg'] = train_client['client_catg'].astype('object')
train_client['disrict'] = train_client['disrict'].astype('object')

test_client['client_catg'] = test_client['client_catg'].astype('object')
test_client['disrict'] = test_client['disrict'].astype('object')

In [ ]:
train_invoice['counter_type'].value_counts()

0    3079406
1    1397343
Name: counter_type, dtype: int64

In [ ]:
for df in [train_invoice,test_invoice]:
    df['invoice_date'] = pd.to_datetime(df['invoice_date'])
    df['year'] = df['invoice_date'].dt.year
    df['month'] = df['invoice_date'].dt.month


In [ ]:
aggs = {}
aggs['consommation_level_1'] = ['sum','max','min','mean','std']
aggs['consommation_level_2'] = ['sum','max','min','mean','std']
aggs['consommation_level_3'] = ['sum','max','min','mean','std']
aggs['consommation_level_4'] = ['sum','max','min','mean','std']

aggs['month'] = ['mean', 'max', 'min', 'std']
aggs['year'] = ['nunique','max','min','mean']

aggs['months_number'] = ['max','min','mean','sum']
aggs['reading_remarque'] = ['max','min','mean','std','sum']
aggs['counter_coefficient'] = ['max', 'min','mean']
aggs['counter_number'] = ['nunique']
aggs['counter_type'] = ['nunique','mean','sum']
aggs['counter_statue'] = ['nunique']
aggs['tarif_type'] = ['nunique','max','min']
aggs['counter_code'] = ['nunique','max','mean','min']


aggs['old_index'] = ['nunique','mean','std']
aggs['old_index'] = ['nunique','mean','std']





In [ ]:
    agg_trans = train_invoice.groupby(['client_id']).agg(aggs)
    agg_trans.columns = ['_'.join(col).strip() for col in agg_trans.columns.values]
    agg_trans.reset_index(inplace=True)

    df = (train_invoice.groupby('client_id')
          .size()
          .reset_index(name='{}transactions_count'.format('1')))

    agg_trans = pd.merge(df, agg_trans, on='client_id', how='left')

In [ ]:
agg_trans.head()

,client_id,1transactions_count,consommation_level_1_sum,consommation_level_1_max,consommation_level_1_min,consommation_level_1_mean,consommation_level_1_std,consommation_level_2_sum,consommation_level_2_max,consommation_level_2_min,consommation_level_2_mean,consommation_level_2_std,consommation_level_3_sum,consommation_level_3_max,consommation_level_3_min,consommation_level_3_mean,consommation_level_3_std,consommation_level_4_sum,consommation_level_4_max,consommation_level_4_min,consommation_level_4_mean,consommation_level_4_std,month_mean,month_max,month_min,month_std,year_nunique,year_max,year_min,year_mean,months_number_max,months_number_min,months_number_mean,months_number_sum,reading_remarque_max,reading_remarque_min,reading_remarque_mean,reading_remarque_std,reading_remarque_sum,counter_coefficient_max,counter_coefficient_min,counter_coefficient_mean,counter_number_nunique,counter_type_nunique,counter_type_mean,counter_type_sum,counter_statue_nunique,tarif_type_nunique,tarif_type_max,tarif_type_min,counter_code_nunique,counter_code_max,counter_code_mean,counter_code_min,old_index_nunique,old_index_mean,old_index_std
0,train_Client_0,35,12334,1200,38,352.400000,310.343472,370,186,0,10.571429,43.568935,0,0,0,0.000000,0.000000,0,0,0,0.000000,0.000000,6.685714,12,1,3.587326,15,2019,2005,2011.142857,12,2,4.628571,162,9,6,6.971429,1.248192,244,1,1,1.0,1,1,0.0,0,1,1,11,11,2,207,203.685714,203,35,9444.542857,4527.744415
1,train_Client_1,37,20629,1207,190,557.540541,197.935960,0,0,0,0.000000,0.000000,0,0,0,0.000000,0.000000,0,0,0,0.000000,0.000000,5.135135,10,1,3.128384,15,2019,2005,2011.837838,8,2,4.324324,160,9,6,7.216216,1.377097,267,1,1,1.0,1,1,0.0,0,1,1,11,11,1,203,203.000000,203,37,14330.756757,6124.126094
2,train_Client_10,18,14375,2400,188,798.611111,513.841374,682,682,0,37.888889,160.748942,0,0,0,0.000000,0.000000,0,0,0,0.000000,0.000000,6.611111,11,2,3.292396,12,2019,2005,2010.944444,12,4,6.444444,116,9,6,7.055556,1.258955,127,1,1,1.0,1,1,0.0,0,1,1,11,11,2,207,203.222222,203,18,32873.000000,5317.180865
3,train_Client_100,20,24,15,0,1.200000,3.607011,0,0,0,0.000000,0.000000,0,0,0,0.000000,0.000000,0,0,0,0.000000,0.000000,5.550000,10,1,3.203206,8,2012,2005,2008.600000,6,4,4.200000,84,9,6,6.150000,0.670820,123,1,1,1.0,1,1,0.0,0,1,1,11,11,1,413,413.000000,413,4,94.150000,4.120232
4,train_Client_1000,14,9292,800,124,663.714286,224.831365,1468,400,0,104.857143,167.155320,1643,800,0,117.357143,289.433294,514,382,0,36.714286,105.421081,7.571429,12,2,3.390355,5,2019,2015,2016.857143,4,2,3.714286,52,9,8,8.857143,0.363137,124,1,1,1.0,1,1,0.0,0,1,1,11,11,1,207,207.000000,207,14,6900.428571,4732.111217


In [ ]:
train = pd.merge(train_client,agg_trans, on='client_id', how='left')

In [ ]:
test_client.head()

,disrict,client_id,client_catg,region,creation_date
0,62,test_Client_0,11,307,28/05/2002
1,69,test_Client_1,11,103,06/08/2009
2,62,test_Client_10,11,310,07/04/2004
3,60,test_Client_100,11,101,08/10/1992
4,62,test_Client_1000,11,301,21/07/1977


In [ ]:
test_invoice.head()

,client_id,invoice_date,tarif_type,counter_number,counter_statue,counter_code,reading_remarque,counter_coefficient,consommation_level_1,consommation_level_2,consommation_level_3,consommation_level_4,old_index,new_index,months_number,counter_type,year,month
0,test_Client_0,2018-03-16,11,651208,0,203,8,1,755,0,0,0,19145,19900,8,ELEC,2018,3
1,test_Client_0,2014-03-21,11,651208,0,203,8,1,1067,0,0,0,13725,14792,8,ELEC,2014,3
2,test_Client_0,2014-07-17,11,651208,0,203,8,1,0,0,0,0,14792,14792,4,ELEC,2014,7
3,test_Client_0,2015-07-13,11,651208,0,203,9,1,410,0,0,0,16122,16532,4,ELEC,2015,7
4,test_Client_0,2016-07-19,11,651208,0,203,9,1,412,0,0,0,17471,17883,4,ELEC,2016,7


In [ ]:
d={"ELEC":0,"GAZ":1}
test_invoice['counter_type']=test_invoice['counter_type'].map(d)

In [ ]:
    agg_trans = test_invoice.groupby(['client_id']).agg(aggs)
    agg_trans.columns = ['_'.join(col).strip() for col in agg_trans.columns.values]
    agg_trans.reset_index(inplace=True)

    df = (test_invoice.groupby('client_id')
          .size()
          .reset_index(name='{}transactions_count'.format('1')))

    agg_trans = pd.merge(df, agg_trans, on='client_id', how='left')

In [ ]:
test = pd.merge(test_client,agg_trans, on='client_id', how='left')

In [ ]:
train.head(50)


,disrict,client_id,client_catg,region,creation_date,target,1transactions_count,consommation_level_1_sum,consommation_level_1_max,consommation_level_1_min,consommation_level_1_mean,consommation_level_1_std,consommation_level_2_sum,consommation_level_2_max,consommation_level_2_min,consommation_level_2_mean,consommation_level_2_std,consommation_level_3_sum,consommation_level_3_max,consommation_level_3_min,consommation_level_3_mean,consommation_level_3_std,consommation_level_4_sum,consommation_level_4_max,consommation_level_4_min,consommation_level_4_mean,consommation_level_4_std,month_mean,month_max,month_min,month_std,year_nunique,year_max,year_min,year_mean,months_number_max,months_number_min,months_number_mean,months_number_sum,reading_remarque_max,reading_remarque_min,reading_remarque_mean,reading_remarque_std,reading_remarque_sum,counter_coefficient_max,counter_coefficient_min,counter_coefficient_mean,counter_number_nunique,counter_type_nunique,counter_type_mean,counter_type_sum,counter_statue_nunique,tarif_type_nunique,tarif_type_max,tarif_type_min,counter_code_nunique,counter_code_max,counter_code_mean,counter_code_min,old_index_nunique,old_index_mean,old_index_std
0,60,train_Client_0,11,101,31/12/1994,0.0,35,12334,1200,38,352.400000,310.343472,370,186,0,10.571429,43.568935,0,0,0,0.000000,0.000000,0,0,0,0.000000,0.000000,6.685714,12,1,3.587326,15,2019,2005,2011.142857,12,2,4.628571,162,9,6,6.971429,1.248192,244,1,1,1.0,1,1,0.000000,0,1,1,11,11,2,207,203.685714,203,35,9444.542857,4527.744415
1,69,train_Client_1,11,107,29/05/2002,0.0,37,20629,1207,190,557.540541,197.935960,0,0,0,0.000000,0.000000,0,0,0,0.000000,0.000000,0,0,0,0.000000,0.000000,5.135135,10,1,3.128384,15,2019,2005,2011.837838,8,2,4.324324,160,9,6,7.216216,1.377097,267,1,1,1.0,1,1,0.000000,0,1,1,11,11,1,203,203.000000,203,37,14330.756757,6124.126094
2,62,train_Client_10,11,301,13/03/1986,0.0,18,14375,2400,188,798.611111,513.841374,682,682,0,37.888889,160.748942,0,0,0,0.000000,0.000000,0,0,0,0.000000,0.000000,6.611111,11,2,3.292396,12,2019,2005,2010.944444,12,4,6.444444,116,9,6,7.055556,1.258955,127,1,1,1.0,1,1,0.000000,0,1,1,11,11,2,207,203.222222,203,18,32873.000000,5317.180865
3,69,train_Client_100,11,105,11/07/1996,0.0,20,24,15,0,1.200000,3.607011,0,0,0,0.000000,0.000000,0,0,0,0.000000,0.000000,0,0,0,0.000000,0.000000,5.550000,10,1,3.203206,8,2012,2005,2008.600000,6,4,4.200000,84,9,6,6.150000,0.670820,123,1,1,1.0,1,1,0.000000,0,1,1,11,11,1,413,413.000000,413,4,94.150000,4.120232
4,62,train_Client_1000,11,303,14/10/2014,0.0,14,9292,800,124,663.714286,224.831365,1468,400,0,104.857143,167.155320,1643,800,0,117.357143,289.433294,514,382,0,36.714286,105.421081,7.571429,12,2,3.390355,5,2019,2015,2016.857143,4,2,3.714286,52,9,8,8.857143,0.363137,124,1,1,1.0,1,1,0.000000,0,1,1,11,11,1,207,207.000000,207,14,6900.428571,4732.111217
5,69,train_Client_10000,11,103,29/09/1993,0.0,48,16901,1414,0,352.104167,356.060910,501,400,0,10.437500,58.447251,181,181,0,3.770833,26.125100,0,0,0,0.000000,0.000000,5.354167,12,1,3.817594,14,2019,2006,2012.583333,12,2,4.000000,192,9,6,7.708333,1.287556,370,1,1,1.0,2,2,0.395833,19,1,2,40,11,2,203,124.625000,5,45,13789.625000,10536.192177
6,62,train_Client_100000,11,309,07/06/2012,0.0,40,11201,1200,0,280.025000,316.711426,262,262,0,6.550000,41.425837,0,0,0,0.000000,0.000000,0,0,0,0.000000,0.000000,7.000000,11,3,3.389274,8,2019,2012,2015.650000,8,2,3.700000,148,9,6,7.975000,1.165476,319,1,1,1.0,2,2,0.500000,20,2,2,40,11,2,413,209.000000,5,37,19788.750000,16818.314522
7,60,train_Client_100001,11,101,12/04/2006,0.0,78,25545,1101,0,327.500000,293.986648,72,64,0,0.923077,7.291285,0,0,0,0.000000,0.000000,0,0,0,0.000000,0.000000,8.846154,12,4,3.162594,14,2019,2006,2012.435897,8,2,3.871795,302,9,6,7.025641,1.309053,548,1,1,1.0,2,2,0.487179,38,2,2,40,11,2,203,106.538462,5,71,6583.141026,6652.875339
8,62,train_Client_100002,11,301,31/12/2006,0.0,2,225,218,7,112.500000,149.199531,0,0,0,0.000000,0.000000,0,0,0,0.000000,0.000000,0,0,0,0.000000,0.000000,

In [ ]:
test.head()

,disrict,client_id,client_catg,region,creation_date,1transactions_count,consommation_level_1_sum,consommation_level_1_max,consommation_level_1_min,consommation_level_1_mean,consommation_level_1_std,consommation_level_2_sum,consommation_level_2_max,consommation_level_2_min,consommation_level_2_mean,consommation_level_2_std,consommation_level_3_sum,consommation_level_3_max,consommation_level_3_min,consommation_level_3_mean,consommation_level_3_std,consommation_level_4_sum,consommation_level_4_max,consommation_level_4_min,consommation_level_4_mean,consommation_level_4_std,month_mean,month_max,month_min,month_std,year_nunique,year_max,year_min,year_mean,months_number_max,months_number_min,months_number_mean,months_number_sum,reading_remarque_max,reading_remarque_min,reading_remarque_mean,reading_remarque_std,reading_remarque_sum,counter_coefficient_max,counter_coefficient_min,counter_coefficient_mean,counter_number_nunique,counter_type_nunique,counter_type_mean,counter_type_sum,counter_statue_nunique,tarif_type_nunique,tarif_type_max,tarif_type_min,counter_code_nunique,counter_code_max,counter_code_mean,counter_code_min,old_index_nunique,old_index_mean,old_index_std
0,62,test_Client_0,11,307,28/05/2002,37,18061,1090,0,488.135135,230.388942,120,120,0,3.243243,19.727878,0,0,0,0.000000,0.000000,0,0,0,0.000000,0.000000,6.837838,12,3,3.337610,15,2019,2005,2011.648649,8,2,4.378378,162,9,6,6.810811,1.221061,252,1,1,1.0,1,1,0.000000,0,1,1,11,11,1,203,203.000000,203,36,11389.837838,5401.043096
1,69,test_Client_1,11,103,06/08/2009,22,24011,3600,11,1091.409091,739.185437,18549,4053,0,843.136364,986.933891,4011,1144,0,182.318182,316.613838,12899,12899,0,586.318182,2750.076040,6.181818,10,1,2.538023,11,2019,2009,2014.136364,12,2,4.545455,100,9,6,7.636364,1.216766,168,1,1,1.0,1,1,0.000000,0,2,1,11,11,1,433,433.000000,433,22,29380.772727,17638.422718
2,62,test_Client_10,11,310,07/04/2004,74,40999,1200,0,554.040541,346.775891,2765,400,0,37.364865,95.536205,1165,800,0,15.743243,101.690110,12,12,0,0.162162,1.394972,7.986486,12,4,3.279213,15,2019,2005,2012.891892,4,4,4.000000,296,9,6,7.459459,1.482216,552,1,1,1.0,2,2,0.432432,32,1,2,40,11,2,207,119.648649,5,74,11774.054054,10987.377541
3,60,test_Client_100,11,101,08/10/1992,40,9774,721,0,244.350000,247.253171,0,0,0,0.000000,0.000000,0,0,0,0.000000,0.000000,0,0,0,0.000000,0.000000,5.700000,12,1,3.450752,8,2012,2005,2008.450000,4,2,3.900000,156,9,6,6.575000,1.034966,263,1,1,1.0,2,2,0.500000,20,1,2,40,11,2,203,104.000000,5,39,13623.725000,13242.361986
4,62,test_Client_1000,11,301,21/07/1977,53,30114,2400,0,568.188679,486.891871,7688,1362,0,145.056604,348.808599,1785,1340,0,33.679245,187.919149,0,0,0,0.000000,0.000000,7.320755,12,2,3.683506,15,2019,2005,2012.622642,12,2,4.528302,240,9,6,7.905660,1.319443,419,1,1,1.0,3,2,0.339623,18,2,2,40,11,2,203,135.754717,5,52,13927.471698,14371.199312


In [ ]:
train.shape,test.shape

((135493, 62), (58069, 61))

In [ ]:
for df in [train,test]:
    df['creation_date'] = pd.to_datetime(df['creation_date'])
    df['year'] = df['creation_date'].dt.year
    df['month'] = df['creation_date'].dt.month
    df['month_diff'] = ((datetime.datetime.today() - df['creation_date']).dt.days)//30

In [ ]:
col_to_drop = ['client_id', 'creation_date','old_index_std','reading_remarque_std','month_std','consommation_level_1_std', 'consommation_level_2_std', 'consommation_level_3_std','consommation_level_4_std']
for col in col_to_drop:
    if col in train.columns:
        train.drop([col], axis=1, inplace=True)
    if col in test.columns:
        test.drop([col], axis=1, inplace=True)

In [ ]:
test.head()

,disrict,client_catg,region,1transactions_count,consommation_level_1_sum,consommation_level_1_max,consommation_level_1_min,consommation_level_1_mean,consommation_level_2_sum,consommation_level_2_max,consommation_level_2_min,consommation_level_2_mean,consommation_level_3_sum,consommation_level_3_max,consommation_level_3_min,consommation_level_3_mean,consommation_level_4_sum,consommation_level_4_max,consommation_level_4_min,consommation_level_4_mean,month_mean,month_max,month_min,year_nunique,year_max,year_min,year_mean,months_number_max,months_number_min,months_number_mean,months_number_sum,reading_remarque_max,reading_remarque_min,reading_remarque_mean,reading_remarque_sum,counter_coefficient_max,counter_coefficient_min,counter_coefficient_mean,counter_number_nunique,counter_type_nunique,counter_type_mean,counter_type_sum,counter_statue_nunique,tarif_type_nunique,tarif_type_max,tarif_type_min,counter_code_nunique,counter_code_max,counter_code_mean,counter_code_min,old_index_nunique,old_index_mean,year,month,month_diff
0,62,11,307,37,18061,1090,0,488.135135,120,120,0,3.243243,0,0,0,0.000000,0,0,0,0.000000,6.837838,12,3,15,2019,2005,2011.648649,8,2,4.378378,162,9,6,6.810811,252,1,1,1.0,1,1,0.000000,0,1,1,11,11,1,203,203.000000,203,36,11389.837838,2002,5,238
1,69,11,103,22,24011,3600,11,1091.409091,18549,4053,0,843.136364,4011,1144,0,182.318182,12899,12899,0,586.318182,6.181818,10,1,11,2019,2009,2014.136364,12,2,4.545455,100,9,6,7.636364,168,1,1,1.0,1,1,0.000000,0,2,1,11,11,1,433,433.000000,433,22,29380.772727,2009,6,152
2,62,11,310,74,40999,1200,0,554.040541,2765,400,0,37.364865,1165,800,0,15.743243,12,12,0,0.162162,7.986486,12,4,15,2019,2005,2012.891892,4,4,4.000000,296,9,6,7.459459,552,1,1,1.0,2,2,0.432432,32,1,2,40,11,2,207,119.648649,5,74,11774.054054,2004,7,212
3,60,11,101,40,9774,721,0,244.350000,0,0,0,0.000000,0,0,0,0.000000,0,0,0,0.000000,5.700000,12,1,8,2012,2005,2008.450000,4,2,3.900000,156,9,6,6.575000,263,1,1,1.0,2,2,0.500000,20,1,2,40,11,2,203,104.000000,5,39,13623.725000,1992,8,357
4,62,11,301,53,30114,2400,0,568.188679,7688,1362,0,145.056604,1785,1340,0,33.679245,0,0,0,0.000000,7.320755,12,2,15,2019,2005,2012.622642,12,2,4.528302,240,9,6,7.905660,419,1,1,1.0,3,2,0.339623,18,2,2,40,11,2,203,135.754717,5,52,13927.471698,1977,7,540


In [ ]:
from sklearn import preprocessing
for f in test.columns:
    if train[f].dtype=='object' or test[f].dtype=='object': 
        lbl = preprocessing.LabelEncoder()
        lbl.fit(list(train[f].values) + list(test[f].values))
        train[f] = lbl.transform(list(train[f].values))
        test[f] = lbl.transform(list(test[f].values))  

In [ ]:
test.head()

,disrict,client_catg,region,1transactions_count,consommation_level_1_sum,consommation_level_1_max,consommation_level_1_min,consommation_level_1_mean,consommation_level_2_sum,consommation_level_2_max,consommation_level_2_min,consommation_level_2_mean,consommation_level_3_sum,consommation_level_3_max,consommation_level_3_min,consommation_level_3_mean,consommation_level_4_sum,consommation_level_4_max,consommation_level_4_min,consommation_level_4_mean,month_mean,month_max,month_min,year_nunique,year_max,year_min,year_mean,months_number_max,months_number_min,months_number_mean,months_number_sum,reading_remarque_max,reading_remarque_min,reading_remarque_mean,reading_remarque_sum,counter_coefficient_max,counter_coefficient_min,counter_coefficient_mean,counter_number_nunique,counter_type_nunique,counter_type_mean,counter_type_sum,counter_statue_nunique,tarif_type_nunique,tarif_type_max,tarif_type_min,counter_code_nunique,counter_code_max,counter_code_mean,counter_code_min,old_index_nunique,old_index_mean,year,month,month_diff
0,1,0,307,37,18061,1090,0,488.135135,120,120,0,3.243243,0,0,0,0.000000,0,0,0,0.000000,6.837838,12,3,15,2019,2005,2011.648649,8,2,4.378378,162,9,6,6.810811,252,1,1,1.0,1,1,0.000000,0,1,1,11,11,1,203,203.000000,203,36,11389.837838,2002,5,238
1,3,0,103,22,24011,3600,11,1091.409091,18549,4053,0,843.136364,4011,1144,0,182.318182,12899,12899,0,586.318182,6.181818,10,1,11,2019,2009,2014.136364,12,2,4.545455,100,9,6,7.636364,168,1,1,1.0,1,1,0.000000,0,2,1,11,11,1,433,433.000000,433,22,29380.772727,2009,6,152
2,1,0,310,74,40999,1200,0,554.040541,2765,400,0,37.364865,1165,800,0,15.743243,12,12,0,0.162162,7.986486,12,4,15,2019,2005,2012.891892,4,4,4.000000,296,9,6,7.459459,552,1,1,1.0,2,2,0.432432,32,1,2,40,11,2,207,119.648649,5,74,11774.054054,2004,7,212
3,0,0,101,40,9774,721,0,244.350000,0,0,0,0.000000,0,0,0,0.000000,0,0,0,0.000000,5.700000,12,1,8,2012,2005,2008.450000,4,2,3.900000,156,9,6,6.575000,263,1,1,1.0,2,2,0.500000,20,1,2,40,11,2,203,104.000000,5,39,13623.725000,1992,8,357
4,1,0,301,53,30114,2400,0,568.188679,7688,1362,0,145.056604,1785,1340,0,33.679245,0,0,0,0.000000,7.320755,12,2,15,2019,2005,2012.622642,12,2,4.528302,240,9,6,7.905660,419,1,1,1.0,3,2,0.339623,18,2,2,40,11,2,203,135.754717,5,52,13927.471698,1977,7,540


In [ ]:
all_data_na = train.isnull().sum() 
all_data_na = all_data_na.drop(all_data_na[all_data_na == 0].index).sort_values(ascending=False)[:30]
missing_data = pd.DataFrame({'Missing ' :all_data_na})
missing_data.head(20)

,Missing


In [ ]:
train.head()

,disrict,client_catg,region,target,1transactions_count,consommation_level_1_sum,consommation_level_1_max,consommation_level_1_min,consommation_level_1_mean,consommation_level_2_sum,consommation_level_2_max,consommation_level_2_min,consommation_level_2_mean,consommation_level_3_sum,consommation_level_3_max,consommation_level_3_min,consommation_level_3_mean,consommation_level_4_sum,consommation_level_4_max,consommation_level_4_min,consommation_level_4_mean,month_mean,month_max,month_min,year_nunique,year_max,year_min,year_mean,months_number_max,months_number_min,months_number_mean,months_number_sum,reading_remarque_max,reading_remarque_min,reading_remarque_mean,reading_remarque_sum,counter_coefficient_max,counter_coefficient_min,counter_coefficient_mean,counter_number_nunique,counter_type_nunique,counter_type_mean,counter_type_sum,counter_statue_nunique,tarif_type_nunique,tarif_type_max,tarif_type_min,counter_code_nunique,counter_code_max,counter_code_mean,counter_code_min,old_index_nunique,old_index_mean,year,month,month_diff
0,0,0,101,0.0,35,12334,1200,38,352.400000,370,186,0,10.571429,0,0,0,0.000000,0,0,0,0.000000,6.685714,12,1,15,2019,2005,2011.142857,12,2,4.628571,162,9,6,6.971429,244,1,1,1.0,1,1,0.0,0,1,1,11,11,2,207,203.685714,203,35,9444.542857,1994,12,328
1,3,0,107,0.0,37,20629,1207,190,557.540541,0,0,0,0.000000,0,0,0,0.000000,0,0,0,0.000000,5.135135,10,1,15,2019,2005,2011.837838,8,2,4.324324,160,9,6,7.216216,267,1,1,1.0,1,1,0.0,0,1,1,11,11,1,203,203.000000,203,37,14330.756757,2002,5,238
2,1,0,301,0.0,18,14375,2400,188,798.611111,682,682,0,37.888889,0,0,0,0.000000,0,0,0,0.000000,6.611111,11,2,12,2019,2005,2010.944444,12,4,6.444444,116,9,6,7.055556,127,1,1,1.0,1,1,0.0,0,1,1,11,11,2,207,203.222222,203,18,32873.000000,1986,3,435
3,3,0,105,0.0,20,24,15,0,1.200000,0,0,0,0.000000,0,0,0,0.000000,0,0,0,0.000000,5.550000,10,1,8,2012,2005,2008.600000,6,4,4.200000,84,9,6,6.150000,123,1,1,1.0,1,1,0.0,0,1,1,11,11,1,413,413.000000,413,4,94.150000,1996,11,305
4,1,0,303,0.0,14,9292,800,124,663.714286,1468,400,0,104.857143,1643,800,0,117.357143,514,382,0,36.714286,7.571429,12,2,5,2019,2015,2016.857143,4,2,3.714286,52,9,8,8.857143,124,1,1,1.0,1,1,0.0,0,1,1,11,11,1,207,207.000000,207,14,6900.428571,2014,10,87


### Models

In [ ]:
target=train['target']
train.drop('target',axis=1,inplace=True)

In [ ]:
test.head()

,disrict,client_catg,region,1transactions_count,consommation_level_1_sum,consommation_level_1_max,consommation_level_1_min,consommation_level_1_mean,consommation_level_2_sum,consommation_level_2_max,consommation_level_2_min,consommation_level_2_mean,consommation_level_3_sum,consommation_level_3_max,consommation_level_3_min,consommation_level_3_mean,consommation_level_4_sum,consommation_level_4_max,consommation_level_4_min,consommation_level_4_mean,month_mean,month_max,month_min,year_nunique,year_max,year_min,year_mean,months_number_max,months_number_min,months_number_mean,months_number_sum,reading_remarque_max,reading_remarque_min,reading_remarque_mean,reading_remarque_sum,counter_coefficient_max,counter_coefficient_min,counter_coefficient_mean,counter_number_nunique,counter_type_nunique,counter_type_mean,counter_type_sum,counter_statue_nunique,tarif_type_nunique,tarif_type_max,tarif_type_min,counter_code_nunique,counter_code_max,counter_code_mean,counter_code_min,old_index_nunique,old_index_mean,year,month,month_diff
0,1,0,307,37,18061,1090,0,488.135135,120,120,0,3.243243,0,0,0,0.000000,0,0,0,0.000000,6.837838,12,3,15,2019,2005,2011.648649,8,2,4.378378,162,9,6,6.810811,252,1,1,1.0,1,1,0.000000,0,1,1,11,11,1,203,203.000000,203,36,11389.837838,2002,5,238
1,3,0,103,22,24011,3600,11,1091.409091,18549,4053,0,843.136364,4011,1144,0,182.318182,12899,12899,0,586.318182,6.181818,10,1,11,2019,2009,2014.136364,12,2,4.545455,100,9,6,7.636364,168,1,1,1.0,1,1,0.000000,0,2,1,11,11,1,433,433.000000,433,22,29380.772727,2009,6,152
2,1,0,310,74,40999,1200,0,554.040541,2765,400,0,37.364865,1165,800,0,15.743243,12,12,0,0.162162,7.986486,12,4,15,2019,2005,2012.891892,4,4,4.000000,296,9,6,7.459459,552,1,1,1.0,2,2,0.432432,32,1,2,40,11,2,207,119.648649,5,74,11774.054054,2004,7,212
3,0,0,101,40,9774,721,0,244.350000,0,0,0,0.000000,0,0,0,0.000000,0,0,0,0.000000,5.700000,12,1,8,2012,2005,2008.450000,4,2,3.900000,156,9,6,6.575000,263,1,1,1.0,2,2,0.500000,20,1,2,40,11,2,203,104.000000,5,39,13623.725000,1992,8,357
4,1,0,301,53,30114,2400,0,568.188679,7688,1362,0,145.056604,1785,1340,0,33.679245,0,0,0,0.000000,7.320755,12,2,15,2019,2005,2012.622642,12,2,4.528302,240,9,6,7.905660,419,1,1,1.0,3,2,0.339623,18,2,2,40,11,2,203,135.754717,5,52,13927.471698,1977,7,540


In [ ]:
import lightgbm
from lightgbm import LGBMRegressor
from lightgbm import LGBMClassifier
model = LGBMClassifier(boosting_type='gbdt',num_iteration=500, silent=True)

#Fit to training data
%time model.fit(train,target)

CPU times: user 42.2 s, sys: 237 ms, total: 42.4 s
Wall time: 29.5 s


LGBMClassifier(num_iteration=500)

In [ ]:
import xgboost as xgb
from datetime import datetime
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
clf = xgb.XGBClassifier(
    n_estimators=500,
    max_depth=9,
    learning_rate=0.05,
    subsample=0.9,
    colsample_bytree=0.9,
    random_state=2019,
)
%time clf.fit(train, target)

CPU times: user 6min 58s, sys: 816 ms, total: 6min 59s
Wall time: 7min 3s


XGBClassifier(colsample_bytree=0.9, learning_rate=0.05, max_depth=9,
              n_estimators=500, random_state=2019, subsample=0.9)

In [ ]:
pred=clf.predict_proba(test)
pred

array([[0.9833939 , 0.01660611],
       [0.919286  , 0.08071401],
       [0.9838366 , 0.01616339],
       ...,
       [0.30841374, 0.69158626],
       [0.9981327 , 0.00186728],
       [0.9500115 , 0.04998852]], dtype=float32)

In [ ]:
lgbm_pred=model.predict_proba(test)
lgbm_pred.shape

(58069, 2)

In [ ]:
lgbm_pred[0]

array([0.98503937, 0.01496063])

### ExtraTree clf

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier
model = ExtraTreeClassifier( n_jobs=4, min_samples_split=25,
                            min_samples_leaf=35, max_features=150)
                            
gsc = GridSearchCV(
    estimator=model,
    param_grid={
        #'n_estimators': range(50,126,25),
        'max_features': range(50,401,50),
        'criterion': ['gini','entropy'],
    },
    scoring='accuracy',
    cv=5
)

grid_result = gsc.fit(train, target)

print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Best: 0.944779 using {'criterion': 'gini', 'max_features': 50}


In [ ]:
extratree_clf = ExtraTreeClassifier( criterion= 'gini', max_features=50)
%time extratree_clf.fit(train, target)

CPU times: user 1.42 s, sys: 8.98 ms, total: 1.43 s
Wall time: 1.48 s


ExtraTreeClassifier(max_features=50)

In [ ]:
extratree_pred=extratree_clf.predict_proba(test)
extratree_pred.shape

(58069, 2)

### RF

In [ ]:
rf_clf = RandomForestClassifier(criterion= 'gini', max_features=30)
%time rf_clf.fit(train, target)

CPU times: user 3min 22s, sys: 265 ms, total: 3min 22s
Wall time: 3min 24s


RandomForestClassifier(max_features=30)

In [ ]:
rf_pred=rf_clf.predict_proba(test)
rf_pred.shape

(58069, 2)

In [ ]:
rf_pred

array([[1.  , 0.  ],
       [0.81, 0.19],
       [0.99, 0.01],
       ...,
       [0.46, 0.54],
       [1.  , 0.  ],
       [0.98, 0.02]])

### KNN

In [ ]:
knn_clf = KNeighborsClassifier()
%time knn_clf.fit(train, target)

CPU times: user 49.1 ms, sys: 6.03 ms, total: 55.2 ms
Wall time: 67.7 ms


KNeighborsClassifier()

In [ ]:
knn_pred=knn_clf.predict_proba(test)
knn_pred.shape

KeyboardInterrupt: ignored

In [ ]:
knn_pred

array([[1. , 0. ],
       [0.8, 0.2],
       [1. , 0. ],
       ...,
       [0.6, 0.4],
       [1. , 0. ],
       [0.8, 0.2]])

### DecisionTreeClassifier

In [ ]:
dtree_clf = DecisionTreeClassifier()
%time dtree_clf.fit(train, target)

CPU times: user 4.62 s, sys: 21.8 ms, total: 4.64 s
Wall time: 4.63 s


DecisionTreeClassifier()

In [ ]:
dtree_pred=dtree_clf.predict_proba(test)
dtree_pred.shape

(58069, 2)

In [ ]:
dtree_pred

NameError: ignored

### SVC

In [ ]:
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
svc_clf = make_pipeline(StandardScaler(), SVC(gamma='auto')) #Faster computation
%time svc_clf.fit(train, target)

In [ ]:
svc_pred=svc_clf.predict_proba(test)
svc_pred.shape

In [ ]:
svc_pred

### AdaBoostClassifier

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
ada_clf = AdaBoostClassifier() #Faster computation
%time ada_clf.fit(train, target)

CPU times: user 20.4 s, sys: 105 ms, total: 20.5 s
Wall time: 20.5 s


AdaBoostClassifier()

In [ ]:
ada_pred=ada_clf.predict_proba(test)
ada_pred.shape

(58069, 2)

In [ ]:
ada_pred

array([[0.51849911, 0.48150089],
       [0.51210667, 0.48789333],
       [0.51369718, 0.48630282],
       ...,
       [0.50871273, 0.49128727],
       [0.52730938, 0.47269062],
       [0.5098782 , 0.4901218 ]])

### Result

In [ ]:
preds=pred[:,1]*0.4+0.3*lgbm_pred[:,1]+ 0.1*rf_pred[:,1]+ 0.2*ada_pred[:,1]

In [ ]:
preds = pd.DataFrame(preds)

In [ ]:
preds.head()

,0
0,0.107431
1,0.192499
2,0.108930
3,0.097026
4,0.172219


### Stacking Models - Blending

In [ ]:
!pip install mlxtend

In [ ]:
!pip install mlrose

In [ ]:
import six
import sys
sys.modules['sklearn.externals.six'] = six
import mlrose

In [ ]:
from mlxtend.classifier import StackingCVClassifier

stack_gen = StackingCVClassifier(classifiers=(ada_clf, rf_clf, model),
                                 use_probas=True,
                                 meta_classifier=clf)
%time stack_gen.fit(np.array(train), np.array(target))

CPU times: user 10min 16s, sys: 2.23 s, total: 10min 18s
Wall time: 10min 17s


StackingCVClassifier(classifiers=(AdaBoostClassifier(),
                                  RandomForestClassifier(max_features=30),
                                  LGBMClassifier(num_iteration=500)),
                     meta_classifier=StackingClassifier(estimators=[('rf',
                                                                     RandomForestClassifier(n_estimators=10,
                                                                                            random_state=42)),
                                                                    ('svr',
                                                                     Pipeline(steps=[('standardscaler',
                                                                                      StandardScaler()),
                                                                                     ('linearsvc',
                                                                                      LinearSVC(random_state=42))]))],
 

In [ ]:
stack_gen_pred=stack_gen.predict_proba(test)
stack_gen_pred.shape

(58069, 2)

In [ ]:
stack_gen_pred

array([[0.97425648, 0.02574352],
       [0.92276852, 0.07723148],
       [0.97053287, 0.02946713],
       ...,
       [0.23955546, 0.76044454],
       [0.97790778, 0.02209222],
       [0.96138769, 0.03861231]])

In [ ]:
preds= stack_gen_pred[:,1]
preds = pd.DataFrame(preds)
preds

,0
0,0.025744
1,0.077231
2,0.029467
3,0.021470
4,0.058709
...,...
58064,0.022804
58065,0.029269
58066,0.760445
58067,0.022092


In [ ]:
def blend(X):
    return ((0.1 * rf_clf.predict_proba(X)) + \
            (0.2 * model.predict_proba(X)) + \
            (0.2 * clf.predict_proba(X)) + \
            (0.15 * ada_clf.predict_proba(X)) + \
            (0.35 * stack_gen.predict_proba(np.array(X))))

In [ ]:
preds=pred[:,1]*0.2+0.2*lgbm_pred[:,1]+ 0.1*rf_pred[:,1]+ 0.15*ada_pred[:,1]+ 0.35*stack_gen_pred[:,1]
preds = pd.DataFrame(preds)
preds

,0
0,0.087549
1,0.164448
2,0.090294
3,0.079995
4,0.144206
...,...
58064,0.080634
58065,0.095906
58066,0.639512
58067,0.079315


In [ ]:
submission = pd.DataFrame({
        "client_id": sub["client_id"],
        "target": preds[0]
    })
submission.to_csv('Submission34.csv', index=False)



In [ ]:
#@title Default title text
submission.head(58071)

,client_id,target
0,test_Client_0,0.087549
1,test_Client_1,0.164448
2,test_Client_10,0.090294
3,test_Client_100,0.079995
4,test_Client_1000,0.144206
...,...,...
58064,test_Client_9995,0.080634
58065,test_Client_9996,0.095906
58066,test_Client_9997,0.639512
58067,test_Client_9998,0.079315


In [ ]:
# pred[:,1]*0.3+0.7*lgbm_pred[:,1]
# Best: pred[:,1]*0.4+0.5*lgbm_pred[:,1]+ 0.1*rf_pred[:,1]
# pred[:,1]*0.4+0.4*lgbm_pred[:,1]+ 0.1*rf_pred[:,1]+0.1*knn_pred[:,1] => Bad sub22
# preds=pred[:,1]*0.35+0.35*lgbm_pred[:,1]+ 0.1*rf_pred[:,1]+ 0.2*ada_pred[:,1] => Best (4th)